In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 2.5 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=bc5be053737081b7b724dc75ebf6e58fb51d14f0ac6b643f884c99cccd305643
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv	 heatmap530.ipynb
 a0530.csv	 heatmap531.ipynb
 a0531.csv	 Heat_Map_Online_language.ipynb
 a0601.csv	 __MACOSX
 csv		 non_violant1.csv
 csv.zip	 non_violant2.csv
 dataframe_csv	 violant.csv
 Github		'新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_602 = 'csv/US06_02.csv'

In [8]:
ct = pd.read_csv(ct_path_602)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675
1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399
2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318
3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629
4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne...,https://www.facebook.com/10716057521/posts/101...,419148,1.191176,1.0,39,19,6,1,2,0,14,0,0,0,20,5,13,6,3,9,5,7,0,5,15634,THE WEEK,theweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,All you need to know about everything that mat...,True,2008-02-26 22:32:05,en,0,15634|10157325690307522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'CHINA': 1, 'HONG KONG': 1, 'Russia': 2, 'US'...","{'US': 1, 'Hong Kong’s': 1}"
1,{},"{'US': 1, 'Hong Kong’s': 1}"
2,{'Unsplash': 1},"{'US': 1, 'Hong Kong’s': 1}"
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0602.csv')

In [25]:
ct=pd.read_csv('a0602.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'CHINA': 1, 'HONG KONG': 1, 'Russia': 2, 'US'...","{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
1,{},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
2,{'Unsplash': 1},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
5,{'US': 1},{},{}
8,{},{'Prescott': 1},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0        [HONG KONG, HK, US, China, Russia, Hong Kong, ...
1                                                       []
2                                               [Unsplash]
5                                                     [US]
8                                                       []
                               ...                        
28378                                                   []
28379                                           [St. Paul]
28380                                         [Fort Worth]
28381                             [West Palm Beach Police]
28382                                            [Atlanta]
Name: NER-msg, Length: 19700, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0        [HONG KONG, HK, US, China, Russia, Hong Kong, ...
1                             [Hong Kong’s, US, China, US]
2                   [Unsplash, Hong Kong’s, US, China, US]
5                                                     [US]
8                                               [Prescott]
                               ...                        
28378                                    [Nebraska, Omaha]
28379                       [St. Paul, St. Paul, St. Paul]
28380                             [Fort Worth, Fort Worth]
28381    [West Palm Beach Police, WEST PALM BEACH, West...
28382    [Atlanta, Atlanta, ATLANTA, Ga., Minneapolis, ...
Name: NER, Length: 19700, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['hong kong', 'hk', 'us', 'hong kong’s', 'china', 'russia', 'hong kong', 'america', 'china']),
       list(['china', 'us', 'hong kong’s']),
       list(['unsplash', 'china', 'us', 'hong kong’s']), ...,
       list(['fort worth']),
       list(['west palm beach police', 'west palm beach', 'fla.', 'minneapolis']),
       list(['atlanta', 'atlanta', 'ga.', 'minneapolis', 'america'])],
      dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['hong kong',
 'hk',
 'us',
 'hong kong’s',
 'china',
 'russia',
 'hong kong',
 'america',
 'china',
 'china',
 'us',
 'hong kong’s',
 'unsplash',
 'china',
 'us',
 'hong kong’s',
 'us',
 'prescott',
 'prescott',
 'texas',
 'america',
 'selma',
 'minneapolis',
 'las vegas',
 'harrisonburg',
 'harrisonburg',
 '| salem',
 'salem',
 'minneapolis',
 'chicago',
 'chicago',
 'minnesota',
 'usa',
 'america',
 'usa',
 'broward',
 'broward county',
 'broward',
 'broward county',
 'broward',
 'north attleboro',
 'minneapolis',
 'massachusetts',
 'camden',
 'broward',
 'flint',
 'the united states',
 'flint',
 'shelbyville',
 'camden',
 'broward',
 'the united states',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minnesota',
 'davenport',
 'iowa',
 'davenport',
 'iowa',
 'christ',
 'the united states',
 'south georgia',
 'america',
 'brunswick',
 'minneapolis',
 'louisville',
 'philando castile',
 'the united states',
 'macedonia',
 'macedonia',
 'minneapolis',
 'minnesota',


In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 6392),
 ('minnesota', 2182),
 ('america', 2144),
 ('the united states', 1191),
 ('u.s.', 1147),
 ('us', 988),
 ('houston', 967),
 ('atlanta', 837),
 ('washington', 747),
 ('chicago', 640),
 ('new york', 462),
 ('los angeles', 394),
 ('texas', 360),
 ('new york city', 329),
 ('dc', 325),
 ('las vegas', 310),
 ('philadelphia', 301),
 ('boston', 291),
 ('st. louis', 285),
 ('d.c.', 246),
 ('louisville', 238),
 ('california', 238),
 ('new zealand', 228),
 ('nyc', 211),
 ('seattle', 208),
 ('georgia', 191),
 ('florida', 190),
 ('nj', 184),
 ('dallas', 171),
 ('north carolina', 168),
 ('virginia', 160),
 ('portland', 159),
 ('baltimore', 152),
 ('sacramento', 144),
 ('denver', 143),
 ('michigan', 131),
 ('asbury park', 131),
 ('va.', 129),
 ('alexandria', 127),
 ('st. paul', 120),
 ('mn', 120),
 ('oakland', 114),
 ('miami', 114),
 ('cicero', 113),
 ('china', 110),
 ('detroit', 109),
 ('paris', 107),
 ('tennessee', 106),
 ('pittsburgh', 105),
 ('asbury', 100),
 ('maryland', 9

In [42]:
dataframe602=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

3166

In [44]:
dataframe602.to_csv('dataframe0602.csv')